# 傾向スコアを用いた逆確率重み付け法（IPTW）の実装

In [81]:
import handcalcs.render

In [82]:
# 乱数のシードを設定
import random
import numpy as np

np.random.seed(1234)
random.seed(1234)

In [83]:
# 使用するパッケージ（ライブラリと関数）を定義
# 標準正規分布の生成用
from numpy.random import *

# グラフの描画用
import matplotlib.pyplot as plt

# SciPy 平均0、分散1に正規化（標準化）関数
import scipy.stats

# シグモイド関数をimport
from scipy.special import expit

# scikit-learnから線形回帰をimport
from sklearn.linear_model import LinearRegression

# その他
import pandas as pd

# データの作成

In [84]:
%%render symbolic
# データ数
num_data = 200
# 年齢:15から75歳の一様乱数
x_1 = randint(15, 76, num_data)
# 性別（0を女性、1を男性とします）:0か1の乱数
x_2 = randint(0, 2, num_data)

<IPython.core.display.Latex object>

# テレビCMを見たかどうか

In [85]:
%%render symbolic
# ノイズの生成
e_z = randn(num_data)

# シグモイド関数に入れる部分
z_base = x_1 + (1-x_2)*10 - 40 + 5*e_z

# シグモイド関数を計算
z_prob = expit(0.1*z_base)

# テレビCMを見たかどうかの変数（0は見ていない、1は見た）
Z = np.array([])

for i in range(num_data):
    Z_i = np.random.choice(2, size=1, p=[1-z_prob[i], z_prob[i]])[0]
    Z = np.append(Z, Z_i)

<IPython.core.display.Latex object>

# 購入量Yを作成

In [86]:
%%render symbolic
# ノイズの生成
e_y = randn(num_data)
Y = -x_1 + 30*x_2 + 10*Z + 80 + 10*e_y

<IPython.core.display.Latex object>

# データをまとめた表を作成し、平均値を比べる

In [87]:
df = pd.DataFrame({'年齢': x_1,
                   '性別': x_2,
                   'CMを見た': Z,
                   '購入量': Y,
                   })

df.head()  # 先頭を表示

,年齢,性別,CMを見た,購入量
0,62,0,1.0,24.464285
1,34,0,0.0,45.693411
2,53,1,1.0,64.998281
3,68,1,1.0,47.186898
4,27,1,0.0,100.114260


In [88]:
# 平均値を比べる

print(df[df["CMを見た"] == 1.0].mean())
print("--------")
print(df[df["CMを見た"] == 0.0].mean())

年齢       55.836066
性別        0.483607
CMを見た     1.000000
購入量      49.711478
dtype: float64
--------
年齢       32.141026
性別        0.692308
CMを見た     0.000000
購入量      68.827143
dtype: float64


# 回帰分析を実施

In [89]:
# 説明変数
X = df[["年齢", "性別", "CMを見た"]]

# 被説明変数（目的変数）
y = df["購入量"]

# 回帰の実施
reg2 = LinearRegression().fit(X, y)

# Z=0の場合
X_0 = X.copy()
X_0["CMを見た"] = 0
Y_0 = reg2.predict(X_0)

# Z=1の場合
X_1 = X.copy()
X_1["CMを見た"] = 1
Y_1 = reg2.predict(X_1)

# 傾向スコアの推定

In [90]:
# 説明変数
X = df[["年齢", "性別"]]

# 被説明変数（目的変数）
Z = df["CMを見た"]

# 回帰の実施
reg = LogisticRegression().fit(X, Z)

# 傾向スコアを求める
Z_pre = reg.predict_proba(X)
print(Z_pre[0:5])  # 5人ほどの結果を見てみる

[[0.04002323 0.95997677]
 [0.44525168 0.55474832]
 [0.30065918 0.69934082]
 [0.08101946 0.91898054]
 [0.87013558 0.12986442]]


# 平均処置効果ATEを求める

In [91]:
%%render symbolic
ATE_1_i = Y/Z_pre[:, 1]*Z + (1-Z/Z_pre[:, 1])*Y_1
ATE_0_i = Y/Z_pre[:, 0]*(1-Z) + (1-(1-Z)/Z_pre[:, 0])*Y_0
ATE = 1/len(Y)*(ATE_1_i-ATE_0_i).sum()

<IPython.core.display.Latex object>

In [92]:
print("推定したATE", ATE)

推定したATE 9.75277505424846
